# Importing Libraries

In [266]:
from pathlib import Path
from pyntcloud import PyntCloud
from tqdm import tqdm
from rosbags.highlevel import AnyReader
import pandas as pd
import json
import os
import open3d as o3d
import numpy as np
from scipy.spatial import cKDTree
import subprocess
import math
import plotly.graph_objects as go
import plotly.io as pio
import utm

# Data Extraction

### Extractor Function

In [267]:
def read_bag_file(input_bag_path, output_dir, topics, save_as='pcd'):
    
    if not os.path.exists(input_bag_path):
        print(f"Error: The file {input_bag_path} does not exist.")
        return
        
    data_dir = os.path.join(output_dir, 'data')
    os.makedirs(data_dir, exist_ok=True)
    lidar_frames = os.path.join(data_dir, 'frames')
    os.makedirs(lidar_frames, exist_ok=True)
    
    gps_ts, latitudes, longitudes, altitudes = [], [], [], []
    
    with AnyReader([Path(input_bag_path)]) as reader:
        connections = [x for x in reader.connections if x.topic in topics]

        for connection, timestamp, rawdata in tqdm(reader.messages(connections=connections), desc='Processing Bag...'):
            topic_name = connection.topic

            if topic_name == '/rslidar_points':
                msg = reader.deserialize(rawdata, connection.msgtype)
                points_data = np.frombuffer(msg.data, dtype=np.float32).reshape((msg.height, msg.width, 4))  # Assuming x, y, z, intensity
                
                pcd = points_data[:, :, :3]
                intensities = points_data[:, :, 3]
                flat_pcd = pcd.reshape(-1, 3)
                flat_intensities = intensities.reshape(-1, 1)
                
                # Save to PCD
                df = pd.DataFrame(data=np.hstack((flat_pcd, flat_intensities)), columns=['x', 'y', 'z', 'intensity'])
                cloud = PyntCloud(df)
                ply_file = os.path.join(lidar_frames, f"{timestamp}.ply")
                
                if save_as == 'pcd':
                    pcd_file = os.path.join(lidar_frames, f"{timestamp}.pcd")
                    ply_to_pcd(ply_file, pcd_file)
                    os.remove(ply_file)
                    
                elif save_as == 'ply':
                    cloud.to_file(ply_file)
                
            elif topic_name == '/gnss':
                msg = reader.deserialize(rawdata, connection.msgtype)
                gps_ts.append(timestamp)
                latitudes.append(msg.latitude)
                longitudes.append(msg.longitude)
                altitudes.append(msg.altitude)


    # GPS data to JSON
    gps_data = {
        "timestamps": gps_ts,
        "latitude": latitudes,
        "longitude": longitudes,
        "altitude": altitudes
    }
    gps_json_path = os.path.join(data_dir, "gnss.json")
    with open(gps_json_path, 'w') as f:
        json.dump(gps_data, f, indent=4)
    
    print(f"Lidar frames saved in directory: {lidar_frames}")
    print(f"GPS data saved at: {gps_json_path}")
    
    
    return gps_json_path, lidar_frames
    

### PLY to PCD Converter

In [268]:
def ply_to_pcd(ply_file, pcd_file):
    # Read the PLY file data
    ply_data = PlyData.read(ply_file)
    vertex_data = ply_data['vertex'].data

    # Extract coordinate and intensity data
    x = vertex_data['x']
    y = vertex_data['y']
    z = vertex_data['z']
    intensity = vertex_data['intensity']

    # Stack these arrays into a single numpy array
    data = np.stack((x, y, z, intensity), axis=-1)
    num_points = data.shape[0]

    # Prepare the PCD file header
    header = f"""# .PCD v0.7 - Point Cloud Data file format
VERSION 0.7
FIELDS x y z intensity
SIZE 4 4 4 4
TYPE F F F F
COUNT 1 1 1 1
WIDTH {num_points}
HEIGHT 1
VIEWPOINT 0 0 0 1 0 0 0
POINTS {num_points}
DATA ascii
"""

    # Write the header and data to the PCD file
    with open(pcd_file, 'w') as f:
        f.write(header)
        np.savetxt(f, data, fmt='%f %f %f %f')

### Input Bag

In [269]:
input_bag = '/home/ty/Downloads/AAI/Bags/hamburg.bag'
if os.path.exists(input_bag):
    bag_name = os.path.splitext(os.path.basename(input_bag))[0]
    main_dir = os.path.join(os.path.dirname(input_bag), bag_name)
    os.makedirs(main_dir, exist_ok=True)

In [270]:
gps_data, frames_dir = read_bag_file(input_bag, main_dir,
                                     ['/rslidar_points', '/gnss'],
                                    save_as='ply')

Processing Bag...: 266it [00:02, 104.87it/s]

Lidar frames saved in directory: /home/ty/Downloads/AAI/Bags/hamburg/data/frames
GPS data saved at: /home/ty/Downloads/AAI/Bags/hamburg/data/gnss.json


# Preprocessing

In [271]:
def read_point_cloud(file_or_cloud):
    if isinstance(file_or_cloud, PyntCloud):
        return file_or_cloud
    elif isinstance(file_or_cloud, str):
        cloud = PyntCloud.from_file(file_or_cloud)
        return cloud
    else:
        raise ValueError("Input must be a file path (str) or a Point Cloud object.")

### Playground Functions

In [272]:
def pyntcloud_to_open3d(file_or_cloud):
    cloud = read_point_cloud(file_or_cloud)
    cloud_pts = cloud.points[['x', 'y', 'z', 'intensity']].values
    
    xyz = cloud_pts[:, :3]
    open3d_cloud = o3d.geometry.PointCloud()
    open3d_cloud.points = o3d.utility.Vector3dVector(xyz)
    return open3d_cloud

In [273]:
def color_pcd(file, clr='r'):
    cloud = read_point_cloud(file)
    pcd = pyntcloud_to_open3d(cloud)
    
    if clr == 'r':
        color = [1, 0, 0]
        pcd.paint_uniform_color(color)
    elif clr == 'g':
        color = [0, 1, 0]
        pcd.paint_uniform_color(color)
    elif clr == 'b':
        color = [0, 0, 1]
        pcd.paint_uniform_color(color)
        
    return pcd

In [274]:
def visualize_pcd(pcd):
    o3d.visualization.draw_geometries([pcd])

In [275]:
def visualize_multiple_pcds(*pcds):
    clouds = [pcd for pcd in pcds]
    o3d.visualization.draw_geometries(clouds)

### Basic Preprocessing

In [276]:
def remove_nans(file):
    pcd = read_point_cloud(file)
    points_df = pcd.points
    points_df.dropna(subset=['x', 'y', 'z'], inplace=True)
    return PyntCloud(points_df)

In [277]:
def remove_ego(file, POG_z=-2.9):
    pcd = read_point_cloud(file)
    points_df = pcd.points
    points = points_df.values
    # EGO Dimensions
    ego_length, ego_width, ego_height = 4.27, 2.02, 1.45

    lidar_from_front = ego_length * 0.62
    lidar_from_rear = ego_length - lidar_from_front
    lidar_from_right = ego_width / 2
    lidar_roof_clearance = POG_z + ego_height

    # Masking
    points = np.asarray(pcd.points)
    x_mask = np.logical_or(points[:, 0] < -lidar_from_rear, 
                           points[:, 0] > lidar_from_front)
    y_mask = np.logical_or(points[:, 1] < -(lidar_from_right),
                          points[:, 1] > lidar_from_right)
    z_mask = np.logical_or(points[:, 2] < POG_z,
                          points[:, 2] > 0)
    
    ego_mask = np.logical_or.reduce((x_mask, y_mask, z_mask))

    # Filtering points using the mask and keeping the intensity
    inlier_points = points_df[ego_mask]
    outlier_points = points_df[~ego_mask]

    # Convert filtered DataFrames back to PyntClouds, including intensity data
    inlier_cloud = PyntCloud(inlier_points)
    outlier_cloud = PyntCloud(outlier_points)

    return inlier_cloud, outlier_cloud


In [278]:
def apply_basic_preprocessing(input_dir, output_dir, filter_ego=True):
    sorted_frames = sorted([os.path.join(input_dir, frame) for frame in os.listdir(input_dir) if frame.endswith(".ply")])
    for file in tqdm(sorted_frames, desc='Basic Preprocessing', total=len(sorted_frames)):
        name = os.path.splitext(os.path.basename(file))[0]
        processed_file = os.path.join(output_dir, name+'.ply')
        clean_cloud = remove_nans(file)
        if filter_ego:
            filtered_cloud, _ = remove_ego(clean_cloud)
            filtered_cloud.to_file(processed_file)
        else:
            clean_cloud.to_file(processed_file)
    print(f"Basic Preprocessing Completed! Frames dumped at {output_dir}")

### Advanced Preprocessing (Noise Filtration)

In [279]:
def statistical_outlier_removal(file, k_nbs, z_thresh):
    cloud = read_point_cloud(file)
    # Extract only the spatial coordinates for KDTree calculations
    spatial_points = np.array(cloud.points[['x', 'y', 'z']])

    # Create a KDTree for efficient nearest neighbor search
    kdtree = cKDTree(spatial_points)
    distances, _ = kdtree.query(spatial_points, k=k_nbs + 1)

    # Compute mean and standard deviation of distances to k nearest neighbors
    mean_distances = np.mean(distances[:, 1:], axis=1)  # exclude the first distance (self)
    std_dev = np.std(mean_distances)
    mean_of_mean_distances = np.mean(mean_distances)

    # Filter points based on the mean distance within standard deviation threshold
    mask = mean_distances <= (mean_of_mean_distances + z_thresh * std_dev)
    outlier_mask = mean_distances > (mean_of_mean_distances + z_thresh * std_dev)

    # Use masks to separate inliers and outliers, including all columns
    inlier_points_df = cloud.points[mask]
    outlier_points_df = cloud.points[outlier_mask]

    inlier_cloud = PyntCloud(inlier_points_df)
    outlier_cloud = PyntCloud(outlier_points_df)
    
    return inlier_cloud, outlier_cloud

In [280]:
def z_filter(file, z_range):
    z_min, z_max = z_range
    pcd = read_point_cloud(file)
    points_df = pcd.points
    points = points_df.values
    
    z_axis = points[:, 2]
    z_mask = np.logical_and(z_axis >= z_min, z_axis <= z_max)
    
    # Filtering inlier points based on the mask
    inlier_points_df = points_df[z_mask]
    outlier_points_df = points_df[~z_mask]
    
    # Convert filtered DataFrames back to PyntClouds
    inlier_cloud = PyntCloud(inlier_points_df)
    outlier_cloud = PyntCloud(outlier_points_df)
    
    return inlier_cloud, outlier_cloud

In [281]:
def apply_advanced_preprocessing(input_dir, 
                                 params, apply_SOR=True, 
                                 apply_z_filtering=True):
    k_nbs, thresh = params['SOR']
    z_min, z_max = params['z_filter']
    
    sorted_frames = sorted([os.path.join(input_dir, frame) for frame in os.listdir(input_dir) if frame.endswith(".ply")])
    
    for file in tqdm(sorted_frames, desc='Advanced Preprocessing', total=len(sorted_frames)):
        if apply_SOR:
            filtered_cloud, _ = statistical_outlier_removal(file, k_nbs, thresh)
            
        if apply_z_filtering:
            filtered_cloud, _ = z_filter(filtered_cloud, [z_min, z_max])
            
        filtered_cloud.to_file(file)
        
    print(f"Advanced Preprocessing Completed! Frames dumped at {input_dir}")
    
    

In [282]:
preprocess = True
basic_preprocessing = True
advanced_preprocessing = False
advanced_params = {'SOR':[10, 3],
                   'z_filter':[-2.9, 8]}
                 

In [283]:
if preprocess:
    data_dir = os.path.dirname(frames_dir)
    preprocessed_frames = os.path.join(data_dir, 'preprocessed_frames')
    os.makedirs(preprocessed_frames, exist_ok=True)
    
    if basic_preprocessing:
        apply_basic_preprocessing(frames_dir, preprocessed_frames, filter_ego=True)
        basic_preprocessing = False
    if advanced_preprocessing:
        apply_advanced_preprocessing(preprocessed_frames,
                                    advanced_params, apply_SOR=True,
                                    apply_z_filtering=True)


Basic Preprocessing: 100%|████████████████████| 190/190 [00:02<00:00, 66.91it/s]

Basic Preprocessing Completed! Frames dumped at /home/ty/Downloads/AAI/Bags/hamburg/data/preprocessed_frames


# Lidar Odometry

In [284]:
def run_kiss_icp_pipeline(directory_path, output_directory, parameters, apply_parameters=False):
    
    maxRange = str(parameters['maxRange'])
    current_dir = os.getcwd()

    output_dir = os.path.join(output_directory, 'kiss_icp_results')
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    os.chdir(output_dir)
    if apply_parameters:
        command = [
            "kiss_icp_pipeline",
            directory_path,
            '--max_range',
            maxRange,
            '--deskew'
        ]
        print(f"Using command {command} with settings:\n {parameters}\n apply_parameters: {apply_parameters}")
    
    else:
        command = [
            "kiss_icp_pipeline",
            directory_path
        ]
        print(f"Using command {command} with settings:\n {parameters}\n apply_parameters: {apply_parameters}")
    try:
        subprocess.run(command)
    except Exception as e:
        print(f"Error Running kiss_icp_pipeline: {e}")
    os.chdir(current_dir)
    return output_dir

In [285]:
compute_odometry = True
maxRange = 150

odom_params = {'maxRange':maxRange}

In [286]:
if compute_odometry:
    odom_dir = os.path.join(main_dir, 'odometry')
    os.makedirs(odom_dir, exist_ok=True)
    lidar_odom = run_kiss_icp_pipeline(preprocessed_frames,
                                      odom_dir, odom_params,
                                      apply_parameters=True)

Using command ['kiss_icp_pipeline', '/home/ty/Downloads/AAI/Bags/hamburg/data/preprocessed_frames', '--max_range', '150', '--deskew'] with settings:
 {'maxRange': 150}
 apply_parameters: True


ninja: no work to do.
-- Install configuration: "Release"
-- Up-to-date: /home/ty/anaconda3/envs/kiss-icp-env/lib/python3.8/site-packages/./kiss_icp_pybind.cpython-38-x86_64-linux-gnu.so


Running cmake --build & --install in /home/ty/kiss-icp/python/build/cp38-cp38-manylinux_2_31_x86_64
Dataloader Type: GenericDataset
Trying to guess how to read your data: `pip install "kiss-icp[all]"` is required


Processing Frames: 100%|██████████| 190/190 [00:13<00:00, 14.34 frames/s]

Poses saved at /home/ty/Downloads/AAI/Bags/hamburg/odometry/kiss_icp_results/results/2024-06-29_08-24-03/preprocessed_frames_poses

Runtime settings written at: /home/ty/Downloads/AAI/Bags/hamburg/odometry/kiss_icp_results/results/2024-06-29_08-24-03/runtime_settings.yaml
 ─────────────────────────────────── 
             Metric   Value   Units  
 ─────────────────────────────────── 
  Average Frequency    15     Hz     
    Average Runtime    68     ms     
 ─────────────────────────────────── 


# Trajectory Transformation

In [287]:
def load_states(file):
    data = np.load(file)
    translations = []
    for tf in data:
        translation = tf[:3, 3]
        translations.append(translation)
        
    return data, np.array(translations)

def read_gps(file):
    with open(file, 'r') as json_file:
        data = json.load(json_file)
        
    lat = np.array(data['latitude'])
    lon = np.array(data['longitude'])
    alt = np.array(data['altitude'])
    
    data = np.column_stack((lat, lon, alt))
    return data

In [288]:
def convert_global_to_local(gps_data):
    latlon_gps = gps_data[:, :2]
    first_lat, first_lon = latlon_gps[0]
    utm_x, utm_y, zone_num, zone_letter = utm.from_latlon(first_lat, first_lon)
    offset = np.array([utm_x, utm_y])
    utm_mat = []
    for lat, lon in latlon_gps:
        ux, uy, _, _ = utm.from_latlon(lat, lon)
        utm_arr = np.array([ux, uy])
        utm_mat.append(utm_arr)
        
    utm_mat = np.array(utm_mat)
    local_mat = utm_mat - offset
    off_lst = [offset, zone_num, zone_letter]
    
    return local_mat, off_lst

def convert_local_to_global(local_data, offset):
    utm_offset = offset[0]
    zone_num = offset[1]
    zone_letter = offset[2]
    local_data = local_data[:, :2]
    
    global_mat = []
    for x, y in local_data:
        # Add the local offset to the UTM offset
        ux = utm_offset[0] + x
        uy = utm_offset[1] + y
        lat, lon = utm.to_latlon(ux, uy, zone_num, zone_letter)
        latlon_arr = np.array([lat, lon])
        global_mat.append(latlon_arr)
        
    global_mat = np.array(global_mat)
    
    return global_mat
    

In [289]:
def rot_matrix(heading_angle):
    angle_radians = np.radians(heading_angle)
    rotation_matrix = np.array([[np.cos(angle_radians), -np.sin(angle_radians), 0, 0],
                                    [np.sin(angle_radians), np.cos(angle_radians), 0, 0],
                                    [0, 0, 1, 0],
                                    [0, 0, 0, 1]])
    return rotation_matrix

In [290]:
def compute_rotation_based_on_gps(gps_data, odom_data, frame_idx=-1, add_offset=0):
    poses_data = odom_data[:, :2]
    gps_data = gps_data[:, :2]
    
    tangent_gps = gps_data[frame_idx] - gps_data[0]
    tangent_lidar = poses_data[frame_idx] - poses_data[0]

    tangent_gps_norm = tangent_gps / np.linalg.norm(tangent_gps)
    tangent_lidar_norm = tangent_lidar / np.linalg.norm(tangent_lidar)

    dot_product = np.dot(tangent_gps_norm, tangent_lidar_norm)
    dot_product = np.clip(dot_product, -1.0, 1.0)
    angle_radians = np.arccos(dot_product)
    sign = np.sign(tangent_gps[0] * tangent_lidar[1] - tangent_gps[1] * tangent_lidar[0])
    angle_radians = sign * angle_radians
    angle_degrees = np.degrees(angle_radians)
    print(angle_degrees)
    angle_degrees = -1*angle_degrees + add_offset
    
    return angle_degrees


In [291]:
def mapbox_trajectories(traj: list, filename1: str):
    def lat_lng_bounds(latitudes, longitudes):
        min_lat, max_lat = min(latitudes), max(latitudes)
        min_lng, max_lng = min(longitudes), max(longitudes)
        return min_lat, min_lng, max_lat, max_lng

    def map_center_and_zoom(min_lat, min_lng, max_lat, max_lng):
        center_lat = (min_lat + max_lat) / 2
        center_lng = (min_lng + max_lng) / 2

        lat_diff = max_lat - min_lat
        lng_diff = max_lng - min_lng

        zoom_lat = math.log(180 / lat_diff) / math.log(2)
        zoom_lng = math.log(360 / lng_diff) / math.log(2)

        zoom = min(zoom_lat, zoom_lng)
        return center_lat, center_lng, zoom

    traces = []
    all_latitudes = []
    all_longitudes = []

    for t in traj:
        name, mat = t
        latitudes = mat[:, 0]
        longitudes = mat[:, 1]
        all_latitudes.extend(latitudes)
        all_longitudes.extend(longitudes)
        trace = go.Scattermapbox(
            lon=longitudes,
            lat=latitudes,
            mode='markers',
            marker=go.scattermapbox.Marker(size=8),
            name=name
        )
        traces.append(trace)

    min_lat, min_lng, max_lat, max_lng = lat_lng_bounds(all_latitudes, all_longitudes)
    center_lat, center_lng, zoom = map_center_and_zoom(min_lat, min_lng, max_lat, max_lng)

    fig = go.Figure(traces)

    fig.update_layout(
        mapbox=dict(
            # Replace with your Mapbox access token
            accesstoken='pk.eyJ1IjoiaGFyaXNiaW55b3VzYWYiLCJhIjoiY2xsMXpkcGNoMDlqZDNrcDZrbjdrNHZnNyJ9.nXLa2DYKp8vNfzxbJRokAQ',
            center=dict(lat=center_lat, lon=center_lng),
            zoom=zoom,
            style='satellite'
        ),
        margin=dict(l=0, r=0, t=0, b=0),
        legend=dict(x=0, y=1, bgcolor='rgba(255, 255, 255, 0.5)')
    )
    pio.write_html(fig, file=filename1)

In [292]:
def apply_transformation(gps_data, poses, translations, params, output_file):
    frame_idx = params['frame_index']
    add_offset = params['offset']
    rotate_via = params['use_heading_from']
#     print(rotate_via)
    manual_heading = params['manual_heading']
    
    transformed_poses = []
    transformed_translations = []
    
    if rotate_via == 'gps':
        angle = compute_rotation_based_on_gps(gps_data, translations, frame_idx, add_offset)
#         print(angle)
        r_matrix = rot_matrix(angle)
#         print(r_matrix)
    elif rotate_via == 'manual':
        r_matrix = rot_matrix(manual_heading)
        
    for pose in poses:
        rotated_pose = np.dot(r_matrix, pose)
        translation = rotated_pose[:3, 3]
        transformed_poses.append(rotated_pose)
        transformed_translations.append(translation)
        
    tf_poses = np.array(transformed_poses)
    tf_translations = np.array(transformed_translations)
    np.save(output_file, tf_poses)
    print(f"Transformed poses saved at: {output_file}")
    return tf_poses, tf_translations
            

In [293]:
def plot_trajectories(gps, odom, output_file):
    local_gps, offset = convert_global_to_local(gps)
    global_odom = convert_local_to_global(odom, offset)
#     # Debug statements
#     print("Local GPS data:")
#     print(local_gps)
#     print("Global Odom data:")
#     print(global_odom)
    trajs = [('GPS', gps[:, :2]), ('Lidar', global_odom)]
    mapbox_trajectories(trajs, output_file)

In [294]:
def transform_and_plot_trajectories(params):
    gps_file = os.path.join(data_dir, 'gnss.json')

    for root, dirs, files in os.walk(odom_dir):
        for file in files:
            if file.endswith('.npy'):
                odom_file = os.path.join(root, file)
                break
    
    trajectories = os.path.join(odom_dir, 'trajectories')
    os.makedirs(trajectories, exist_ok=True)
    
    gps_data = read_gps(gps_file)
    poses, translations = load_states(odom_file)
    print(translations[:5])
    traj_file = os.path.join(trajectories, 'transformed_poses.npy')
    local_gps, _ = convert_global_to_local(gps_data)
    tf_poses, tf_translations = apply_transformation(gps_data, poses, translations, params, traj_file)
    print(tf_translations[:5])
    org_plot_file = os.path.join(trajectories, 'lidar_gps_raw.html')
    plot_trajectories(gps_data, translations, org_plot_file)
    print(f"Raw trajectories plot saved at: {org_plot_file}")
    
    tf_plot_file = os.path.join(trajectories, 'lidar_gps_transformed.html')
    plot_trajectories(gps_data, tf_translations, tf_plot_file)
    print(f"Transformed trajectories plot saved at: {tf_plot_file}")
    

In [295]:
params = {'frame_index':-1,
         'offset':0,
         'use_heading_from':'gps',
         'manual_heading': 0
         }

In [296]:
transform_and_plot_trajectories(params)

[[ 0.          0.          0.        ]
 [ 0.01077481  1.16174201 -0.01573739]
 [ 0.03017555  2.88316786 -0.03786862]
 [ 0.0539516   4.37269494 -0.04436926]
 [ 0.05498653  5.4116076  -0.04029995]]
-0.0
Transformed poses saved at: /home/ty/Downloads/AAI/Bags/hamburg/odometry/trajectories/transformed_poses.npy
[[ 0.          0.          0.        ]
 [ 0.01077481  1.16174201 -0.01573739]
 [ 0.03017555  2.88316786 -0.03786862]
 [ 0.0539516   4.37269494 -0.04436926]
 [ 0.05498653  5.4116076  -0.04029995]]
Raw trajectories plot saved at: /home/ty/Downloads/AAI/Bags/hamburg/odometry/trajectories/lidar_gps_raw.html
Transformed trajectories plot saved at: /home/ty/Downloads/AAI/Bags/hamburg/odometry/trajectories/lidar_gps_transformed.html
